
## 1 Importing the necessary libraries

Run the cell below to import the necessary libraries for this assignment.

In [ ]:
from utils import (
    retrieve, 
    pprint, 
    generate_with_single_input, 
    read_dataframe, 
    display_widget
)
import unittests

## 2 - Loading the dataset

In [ ]:
NEWS_DATA = read_dataframe("news_data_dedup.csv")

Let's check the data structure.

In [ ]:
pprint(NEWS_DATA[9:12])

In [ ]:
def query_news(indices):
    """
    Retrieves elements from a dataset based on specified indices.

    Parameters:
    indices (list of int): A list containing the indices of the desired elements in the dataset.
    dataset (list or sequence): The dataset from which elements are to be retrieved. It should support indexing.

    Returns:
    list: A list of elements from the dataset corresponding to the indices provided in list_of_indices.
    """
     
    output = [NEWS_DATA[index] for index in indices]

    return output

In [ ]:
# Fetching some indices
indices = [3, 6, 9]
pprint(query_news(indices))

In [ ]:
# Let's test the retrieve function!
indices = retrieve("Regulations about Autopilot", top_k = 1)
print(indices)

In [ ]:
# Now let's query the corresponding news_
retrieved_documents = query_news(indices)
pprint(retrieved_documents)

In [ ]:
# GRADED CELL 

def get_relevant_data(query: str, top_k: int = 5) -> list[dict]:
    """
    Retrieve and return the top relevant data items based on a given query.

    This function performs the following steps:
    1. Retrieves the indices of the top 'k' relevant items from a dataset based on the provided `query`.
    2. Fetches the corresponding data for these indices from the dataset.

    Parameters:
    - query (str): The search query string used to find relevant items.
    - top_k (int, optional): The number of top items to retrieve. Default is 5.

    Returns:
    - list[dict]: A list of dictionaries containing the data associated 
      with the top relevant items.

    """
    ### START CODE HERE ###

    # Retrieve the indices of the top_k relevant items given the query
    relevant_indices = retrieve(query, top_k)

    # Obtain the data related to the items using the indices from the previous step
    relevant_data = query_news(relevant_indices)

    ### END CODE HERE
    
    return relevant_data

In [ ]:
query = "Greatest storms in the US"
relevant_data = get_relevant_data(query, top_k = 1)
pprint(relevant_data)

In [ ]:
# Run this cell to perform several tests on your function. If you receive "All test passed!" it means that your solution will likely pass the autograder too.
unittests.test_get_relevant_data(get_relevant_data)

## Formatting the relevant Data

 It’s recommended to use double quotes for strings and single quotes for dictionary keys (for example, data['title']). Here’s one way you could format it:

f"""
Title: {news_1_title}, Description: {news_1_description}, Published at: {news_1_published_date}\nURL: {news_1_URL}
Title: {news_2_title}, Description: {news_2_description}, Published at: {news_2_published_date}\nURL: {news_2_URL}
...
Title: {news_k_title}, Description: {news_k_description}, Published at: {news_k_published_date}\nURL: {news_k_URL}
"""

In [60]:
# GRADED CELL

def format_relevant_data(relevant_data):
    """
    Retrieves the top_k most relevant documents based on a given query and constructs an augmented prompt for a RAG system.

    Parameters:
    relevant_data (list): A list with relevant data.

    Returns:
    str: An augmented prompt with the top_k relevant documents, formatted for use in a Retrieval-Augmented Generation (RAG) system."
    """

    ### START CODE HERE ###

    # Create a list so store the formatted documents
    formatted_documents = []
    
    # Iterates over each relevant document.
    for document in relevant_data:

        # Formats each document into a structured layout string. Remember that each document is in one different line. So you should add a new line character after each document added.
        formatted_document = f"Title: {document['title']}, Description: {document['description']}, Published at: {document['published_at']}\nURL: {document['url']}"
        
        # Append the formatted document string to the formatted_documents list
        formatted_documents.append(formatted_document)
    
    ### END CODE HERE ###
    
    # Returns the final augmented prompt string.

    return "\n".join(formatted_documents)

In [61]:
example_data = NEWS_DATA[4:8]
print(format_relevant_data(example_data))

Title: Prosecutors ask for halt to case against Spain PM's wife, Description: Pedro Sánchez is deciding whether to resign after a case against his wife by an anti-corruption group., Published at: 2024-04-25
URL: https://www.bbc.co.uk/news/world-europe-68895727
Title: WATCH: Would you pay a tourist fee to enter Venice?, Description: From Thursday visitors making a trip to the famous city at peak times will be charged a trial entrance fee., Published at: 2024-04-25
URL: https://www.bbc.co.uk/news/world-europe-68898441
Title: Supreme Court divided on whether Trump has immunity, Description: The justices discussed immunity, coups, pardons, Operation Mongoose - and the future of democracy., Published at: 2024-04-25
URL: https://www.bbc.co.uk/news/world-us-canada-68901817
Title: More than 150 killed as heavy rains pound Tanzania, Description: The prime minister warns that El Niño-triggered heavy rains are likely to continue into May., Published at: 2024-04-25
URL: https://www.bbc.co.uk/news/

In [62]:
# Test your function!
unittests.test_format_relevant_data(format_relevant_data)

✅ Test 1: PASSED
✅ Test 2: PASSED
✅ Test 3: PASSED
✅ Test 4: PASSED
✅ Test 5: PASSED
✅ Test 6: PASSED
✅ Test 7: PASSED
✅ Test 8: PASSED

Results: 8/8 tests passed
🎉 All tests passed!
